In [1]:
# Area under curve for finding step frequecny
# Find step frequencies for continuous segments
# Speed find the running speed in continuous intervals

import Data
from IPython.display import clear_output
from tensorflow.keras import layers
from keras import backend as k
import IPython
import numpy as np
import Data
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import model
from model import build_model_CNN
from plotter import hist_plotter
from sklearn.preprocessing import OneHotEncoder

import os

Could not initialize the tensorflow gpu


In [4]:
# Get Data Here
Features_A, Labels, pIDs = Data.dataset_main(150,50,'Yes')
Features_G, Labels, pIDs = Data.dataset_main(150,50,'No')
clear_output()
print('Dataset Loaded')

Dataset Loaded


In [5]:
pIDsUnique = np.unique(pIDs)
pIDsUnique
pIDsInts = np.arange(len(pIDsUnique))
pIDsDict = dict()
for i, ID in enumerate(pIDsUnique):
    pIDsDict[ID] = pIDsInts[i]

pIDsVals = np.array([pIDsDict[ID] for ID in pIDs]).reshape(-1,1)

oneHot = OneHotEncoder(sparse=False)
pIDsEnc = oneHot.fit_transform(pIDsVals)

In [6]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    print('Could not initialize the tensorflow gpu')
    pass

Could not initialize the tensorflow gpu


In [12]:

def R_Square(y_true, y_pred):

    Num = k.sum(k.square(y_true - y_pred))
    Denom = k.sum(k.square(y_true - k.mean(y_true)))
    R = 1 - Num / (Denom + k.epsilon())
    return R


def build_model_CNN():
    inputA = layers.Input(shape=(3, 150, 1))
    modelA = inputA
    min_conv1 = 27
    max_conv1 = 36
    min_conv2 = 36
    max_conv2 = 45
    modelA = layers.Conv2D(27,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(inputA)

    modelA = layers.Conv2D(45,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(modelA)

    #modelA = layers.Flatten()(modelA)
    modelA = layers.GlobalMaxPool2D()(modelA)
    inputG = layers.Input(shape=(3, 150, 1))

    modelG = inputG

    modelG = layers.Conv2D(27,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(inputG)

    modelG = layers.Conv2D(45,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(modelG)

    # model = layers.Dropout(0.4)(model)
    #modelG = layers.Flatten()(modelG)
    modelG = layers.GlobalMaxPool2D()(modelG)
    model = layers.Concatenate()([modelA, modelG])
    #   layers.concatenate(modelA,modelG)

    #model = layers.Dropout(0.4)(model)

    model = layers.Dense(180, activation='relu')(model)

    model = layers.Dropout(0.3)(model)

    model = layers.Dense(30, activation='relu')(model)

    model = layers.Dropout(0.3)(model)

    output = layers.Dense(1)(model)

    model = tf.keras.Model(inputs=[inputA, inputG], outputs=output)

    optimizer1 = tf.keras.optimizers.RMSprop(1e-3)
    # optimizer = tf.keras.optimizers.Adam(1e-3)

    loss1 = tf.keras.losses.MeanAbsoluteError(name="mean_absolute_error")
    
    loss2 = tf.keras.losses.MeanAbsoluteError(name="mean_absolute_error")

    model.compile(
        loss=loss1,  #'mean_absolute_error',
        optimizer=optimizer1,
        metrics=['mae', 'mape', R_Square])

    tf.keras.utils.plot_model(model,
                            #  to_file='/home/redev/Pictures/Model.png',
                              show_shapes=True)
    return model



In [9]:

def R_Square(y_true, y_pred):

    Num = k.sum(k.square(y_true - y_pred))
    Denom = k.sum(k.square(y_true - k.mean(y_true)))
    R = 1 - Num / (Denom + k.epsilon())
    return R


def build_model_CNN():
    inputA = layers.Input(shape=(3, 150, 1))
    modelA = inputA
    min_conv1 = 27
    max_conv1 = 36
    min_conv2 = 36
    max_conv2 = 45
    modelA = layers.Conv2D(27,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(inputA)

    modelA = layers.Conv2D(45,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(modelA)

    #modelA = layers.Flatten()(modelA)
    modelA = layers.GlobalMaxPool2D()(modelA)
    inputG = layers.Input(shape=(3, 150, 1))

    modelG = inputG

    modelG = layers.Conv2D(27,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(inputG)

    modelG = layers.Conv2D(45,
                           kernel_size=(3, 3),
                           padding='same',
                           activation='relu')(modelG)

    # model = layers.Dropout(0.4)(model)
    #modelG = layers.Flatten()(modelG)
    modelG = layers.GlobalMaxPool2D()(modelG)
    model = layers.Concatenate()([modelA, modelG])
    #   layers.concatenate(modelA,modelG)

    #model = layers.Dropout(0.4)(model)

    model = layers.Dense(180, activation='relu')(model)
    #model = layers.Dropout(0.3)(model)
    model = layers.Dense(30, activation='relu')(model)
    #model = layers.Dropout(0.3)(model)
    output = layers.Dense(15)(model)

    model = tf.keras.Model(inputs=[inputA, inputG], outputs=output)

    #optimizer = tf.keras.optimizers.RMSprop(1e-3)
    optimizer = tf.keras.optimizers.Adam(1e-3)

    #loss1 = tf.keras.losses.MeanAbsoluteError(name="mean_absolute_error")
    
    #loss2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    loss2=tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(
        loss=loss2,  #'mean_absolute_error',
        optimizer=optimizer,
        metrics=['accuracy'])

    tf.keras.utils.plot_model(model,
                            #  to_file='/home/redev/Pictures/Model.png',
                              show_shapes=True)
    return model

In [10]:


class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait=True)



earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                 patience=50)

reduceLRplateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                       min_lr=1e-6,
                                                       patience=30,
                                                       factor=0.9,
                                                       verbose=1)

if os.path.isdir('logdir'):
    pass
else:
    os.mkdir('logdir')

logdir = 'logdir'
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)



In [13]:
model = build_model_CNN()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 3, 150, 1)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 3, 150, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 3, 150, 27)   270         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 3, 150, 27)   270         input_4[0][0]                    
____________________________________________________________________________________________

In [21]:
import tensorflow.compat.v2 as tf


class EpochDots(tf.keras.callbacks.Callback):
  """A simple callback that prints a "." every epoch, with occasional reports.
  Args:
    report_every: How many epochs between full reports
    dot_every: How many epochs between dots.
  """

  def __init__(self, report_every=100, dot_every=1):
    self.report_every = report_every
    self.dot_every = dot_every

  def on_epoch_end(self, epoch, logs):
    if epoch % self.report_every == 0:
      print()
      print('Epoch: {:d}, '.format(epoch), end='')
      for name, value in sorted(logs.items()):
        print('{}:{:0.4f}'.format(name, value), end=',  ')
      print()

    if epoch % self.dot_every == 0:
      print('.', end='')

In [22]:
pIDsEnc[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [23]:
X_TrainA, X_TestA, y_TrainA, y_TestA = train_test_split(Features_A, pIDsVals, test_size=0.15, shuffle=True, random_state=90)
X_TrainG, X_TestG, y_TrainG, y_TestG = train_test_split(Features_G, pIDsVals, test_size=0.15, shuffle=True, random_state=90)

X_TrainA, X_ValA, y_TrainA, y_ValA = train_test_split(X_TrainA, y_TrainA, test_size=0.18, shuffle=True, random_state=90)
X_TrainG, X_ValG, y_TrainG, y_ValG = train_test_split(X_TrainG, y_TrainG, test_size=0.18, shuffle=True, random_state=90)

In [24]:
##############This is for classification problem

hist = model.fit(
    [X_TrainA, X_TrainG],
    y_TrainA,
 #   validation_data=([Features_A, Features_G], Labels),
    validation_data = ([X_ValA, X_ValG], y_ValA),
    epochs=1200,
    verbose=0,
    callbacks=[
        EpochDots(report_every=10)
       # ,
       #    earlystopping#,
        #reduceLRplateau
        
      #  tensorboard_callback,
        #ClearTrainingOutput()
    ])

hist_plotter(hist.history)
print("Results of the model training")


Epoch: 0, R_Square:-3.6456,  loss:5.8632,  mae:5.8632,  mape:314859872.0000,  val_R_Square:-0.0946,  val_loss:3.5105,  val_mae:3.5105,  val_mape:308421760.0000,  
..........
Epoch: 10, R_Square:0.4172,  loss:2.3491,  mae:2.3491,  mape:102135840.0000,  val_R_Square:0.6773,  val_loss:1.8519,  val_mae:1.8519,  val_mape:119066208.0000,  
....

KeyboardInterrupt: 

In [ ]:
np.argmax(model.predict([X_TestA,X_TestG]), axis = 1)
model.predict([X_TestA,X_TestG])[:2]

In [ ]:

hist = model.fit(
    [Features_A, Features_G],
    np.array(Labels).reshape(-1,1),
 #   validation_data=([Features_A, Features_G], Labels),
    validation_split = 0.3,
    epochs=50,
    verbose=3,
    callbacks=[
        EpochDots(),
        #        earlystopping,
        reduceLRplateau,
        
      #  tensorboard_callback,
        ClearTrainingOutput()
    ])

hist_plotter(hist.history)
print("Results of the model training")

In [ ]:

hist = model.fit(
    [Features_A, Features_G],
    np.array(Labels).reshape(-1,1),
 #   validation_data=([Features_A, Features_G], Labels),
    validation_split = 0.3,
    epochs=500,
    verbose=3,
    callbacks=[
        #        earlystopping,
        reduceLRplateau,
      #  tensorboard_callback,
        EpochDots(),
        ClearTrainingOutput()
    ])

hist_plotter(hist.history)
print("Results of the model training")

In [ ]:
print(model.evaluate([Features_A, Features_G], np.array(Labels)))

In [9]:
values = np.hstack((np.array(Labels).reshape(-1,1), np.array(pIDs).reshape(-1,1)))
data_df = pd.DataFrame(values, columns = ['speed', 'person'])

In [29]:
# Leave speed out
oneFeat = Features_A[0]

In [34]:
np.fromstring(oneFeat.tostring()).shape

/home/redev/anaconda3/envs/pyHAI/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """Entry point for launching an IPython kernel.


(450,)

In [118]:
FeatureAcol = [feature.tostring() for feature in Features_A]
FeatureGcol = [feature.tostring() for feature in Features_G]
LabelsCol = Labels.ravel()
pIDsCol = pIDsVals.ravel()

dataDF = pd.DataFrame(list(zip(FeatureAcol, FeatureGcol, LabelsCol, pIDsCol)), columns = ['FeatA', 'FeatG', 'Speed', 'ID'])

In [121]:
groupDF = dataDF.groupby(['ID', 'Speed'])

In [135]:
def get_numpyArray(arr):
    return(np.array([np.frombuffer(feat).reshape(3,150,1) for feat in arr]))


def dataParser(dataDF, id, speed):
    print(dataDF.shape)
    
    TrainDF = dataDF[(dataDF['ID']!=id)&(dataDF['Speed']!=speed)]
    TestDF = dataDF[(dataDF['ID']==id)&(dataDF['Speed']==speed)]
    print(TrainDF.shape)
    print(TestDF.shape)
        
    X_TrainA = get_numpyArray(TrainDF['FeatA'])
    X_TrainG = get_numpyArray(TrainDF['FeatG'])
    y_TrainSpeed = TrainDF['Speed'].to_numpy()
    y_TrainID = TrainDF['ID'].to_numpy()
    
    X_TestA = get_numpyArray(TestDF['FeatA'])
    X_TestG = get_numpyArray(TestDF['FeatG'])
    y_TestSpeed = TestDF['Speed'].to_numpy()
    y_TestID = TestDF['ID'].to_numpy()
    
    print(y_TrainSpeed.shape)
    print(y_TrainSpeed.shape)
    print(X_TrainA.shape)
    print(X_TrainG.shape)
        
    X_TrainA, X_ValA, y_TrainA, y_ValSpeed = train_test_split(X_TrainA, y_TrainSpeed, test_size=0.2, shuffle=True, random_state=90)
    X_TrainG, X_ValG, y_TrainG, y_ValSpeed = train_test_split(X_TrainG, y_TrainSpeed, test_size=0.2, shuffle=True, random_state=90)
    
    return(X_TrainA, X_TrainG, y_TrainA, X_TestA, X_TestG, y_TestSpeed, X_ValA, X_ValG,  y_ValSpeed)

In [136]:
results_df = pd.DataFrame(columns = ['id', 'speed','loss', 'mae', 'mape', 'r2','me'])

for id, speed in groupDF.groups:    
    X_TrainA, X_TrainG, y_Train, X_TestA, X_TestG, y_Test, X_ValA, X_ValG, y_Val = dataParser(dataDF, id,speed)

    hist = model.fit(
    [Features_A, Features_G],
    np.array(Labels).reshape(-1,1),
 #   validation_data=([Features_A, Features_G], Labels),
    validation_split = 0.3,
    epochs=5,
    verbose=0,
    callbacks=[
        EpochDots(report_every=10),
        reduceLRplateau,
        ClearTrainingOutput()
    ])

    results = [id, speed]
    results.extend(model.evaluate([X_TestA, X_TestG], np.array(y_Test)))
    results.append(model.predict([X_TestA, X_TestG]).mean())
    
    results_df.loc[len(results_df)] = results
    results=list()

1/1 [==============================] - 0s 979us/step - loss: 0.3583 - mae: 0.3583 - mape: 10.2360 - R_Square: -31766912.0000

(3222, 4)
(2790, 4)
(18, 4)
(2790,)
(2790,)
(2790, 3, 150, 1)
(2790, 3, 150, 1)


KeyboardInterrupt: 